In [1]:
import sys

# Include helper functions in src folder
sys.path.append("../src")

import time
from api_client import create_openai_client, create_google_service, get_openai_response_and_total_tokens, search_google
from helpers import parse_html
from dotenv import dotenv_values
import requests

"""
Main Idea for program
---------------------

1. User submits query.
2. Intelligent LLM "rephraser" rephrases the query.
3. Rephrased query gets submitted to Google.
4. The URL's for the top 5 links are collected.
5. Does SFBLIC come up in the top 5 links?  If not, replace the 5th link with SFBLIC.
6. For each link,
     - the content is fetched 
     - the content is cleaned with Beautiful soup's get_text()
     - a semi-intellignt LLM "refiner" refines the cleaned results so that they contain information relevant to query.
7. Intelligent LLM "writer-citer" is asked to write a response that will answer the refined query using only the cleaned results, providing citations to the results in a certain manner.
9. Visibility metrics are computed.
10. Intelligent LLM "marketer" is asked to re-write or add content to the SFBLIC page according to certain directions.
11. Visibility metrics are computed.
"""

config = dotenv_values("../.env") 

num_results = 5
sfblic_link = "https://www.sfbli.com/"

open_ai_key = config['OPEN_AI_API_KEY']
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Initialize clients
openai_client = create_openai_client(open_ai_key)
google_service, search_id = create_google_service(google_api_key, search_engine_id)

num_tokens_rephrasing = []
num_tokens_refining = []
num_tokens_writing_citing = []
num_tokens_marketing = []

In [2]:
# Step 1: User submits query
original_query = "products offered by southern farm bureau life insurance company"

# Step 2: Rephrase query using OpenAI (LLM name: rephraser)
messages = [
    {"role": "system", "content": "Rephrase the user's query for search engine optimization."},
    {"role": "user", "content": original_query},
]
rephrased_query, total_tokens_from_rephrasing, completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

print(rephrased_query)
print(total_tokens_from_rephrasing)
num_tokens_rephrasing.append(total_tokens_from_rephrasing)

What insurance products does Southern Farm Bureau Life Insurance Company offer?
42


In [4]:
prompt_tokens

30

In [10]:
chat_completion

ChatCompletion(id='chatcmpl-AVdpOx36uHaf71qM2tc9BFMvzz9zQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What insurance products are available from Southern Farm Bureau Life Insurance Company?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732104630, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_831e067d82', usage=CompletionUsage(completion_tokens=13, prompt_tokens=30, total_tokens=43, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [3]:
# Step 3: Refined query gets submitted to Google.
search_results = search_google(google_service, search_id, rephrased_query)

# Step 4: The URL's to the top 5 links are collected.
top_links = [result['link'] for result in search_results[:5]]
top_links

['https://www.sfbli.com/',
 'https://w3.sfbcic.com/',
 'https://www.sfbli.com/loginregister',
 'https://www.kyfb.com/insurance/insurance-products/life-and-financial/',
 'https://msfbins.com/products/life/']

In [4]:
# Step 5: Does SFBLIC come up in the top 5 links?  If not, replace the 5th link with SFBLIC.
if sfblic_link not in top_links:
    top_links = top_links[:4] + [sfblic_link]

top_links

['https://www.sfbli.com/',
 'https://w3.sfbcic.com/',
 'https://www.sfbli.com/loginregister',
 'https://www.kyfb.com/insurance/insurance-products/life-and-financial/',
 'https://msfbins.com/products/life/']

In [5]:
"""
6. For each link,
     - the content is fetched 
     - the content is cleaned with Beautiful soup's get_text()
     - a semi-intellignt LLM "refiner" refines the cleaned results so that they contain information relevant to query.
"""

refined_sources = []

for url in top_links:
    
    time.sleep(1)    

    # The content is fetched    
    response = requests.get(url)

    # The content is cleaned
    raw_text = parse_html(response.content)

    # A semi-intelligent LLM refines the cleaned results so that they contain information relevant to the query
    messages = [
    {"role": "system", "content": f"Refine the following text and extract only relevant information related to the following query: {rephrased_query}"},
    {"role": "user", "content": raw_text},
]

    refined_text, total_tokens_from_refining = get_openai_response_and_total_tokens(openai_client, messages, model="gpt-4o-mini")
    
    print(refined_text)
    print(total_tokens_from_refining)
    num_tokens_refining.append(total_tokens_from_refining)

    refined_sources.append({"url" : url, "refined_text" : refined_text})

Southern Farm Bureau Life Insurance Company offers a variety of insurance products, specifically focusing on life insurance. Further details on specific types of policies or additional products are not provided in the text.
268
The text provided does not contain relevant information regarding the insurance products offered by Southern Farm Bureau Life Insurance Company.
70
Southern Farm Bureau Life Insurance Company offers various insurance products, including:

- Life Insurance
- Variable Life Insurance
- Health Plans
- Behavioral Health Plans

The company aims to provide competitive products and excellent customer service to its policyholders.
440
Southern Farm Bureau Life Insurance Company offers the following insurance products:

1. **Life Insurance**:
   - Term life insurance
   - Whole life insurance

2. **Long-Term Care Insurance**: This provides protection against the costs associated with long-term nursing facility care or home care.

3. **Retirement Planning**: Includes retir

In [6]:
"""
# 7. Intelligent LLM "writer-citer" is asked to write a response that will answer 
the refined query using only the cleaned results, providing citations to the results in a certain manner.
"""

system_content = """
Write an accurate and concise answer for the given user question, 
using _only_ the provided summarized web search results. 
The answer should be correct, high-quality, and written by an expert using an unbiased and journalistic tone. 
The answer should be informative, interesting, and engaging. 
The answer's logic and reasoning should be rigorous and defensible. 
Every sentence in the answer should be _immediately followed_ by an in-line citation to the search result(s). 
The cited search result(s) should fully support _all_ the information in the sentence. 
Search results need to be cited using [index]. 
When citing several search results, use [1][2][3] format rather than [1, 2, 3]. 
You can use multiple search results to respond comprehensively while avoiding irrelevant search results.
"""

messages = [
    {"role": "system", "content": system_content},
    {"role": "user", "content": f"Original Query: {rephrased_query}\n Summarized Web Search Results: {refined_sources}"},
]

final_response, total_tokens_from_final_response = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

In [7]:
print(final_response)

Southern Farm Bureau Life Insurance Company provides a range of insurance products, with a particular emphasis on life insurance. Their life insurance offerings include term life insurance and whole life insurance [3][4]. Additionally, they offer variable life insurance, health plans, and behavioral health plans [2]. The company also provides long-term care insurance, which covers costs for long-term nursing facility care or home care, as well as retirement planning products like annuities, and services for estate planning [4][5]. These products aim to help customers with comprehensive financial planning and protection, and they are accessible through local agents, particularly those associated with the Kentucky Farm Bureau [4].


In [8]:
total_tokens_from_final_response

734

In [9]:
num_tokens_writing_citing.append(total_tokens_from_final_response)

In [10]:
num_tokens_rephrasing 

[42]

In [11]:
num_tokens_refining 

[268, 70, 440, 1374, 99]

In [12]:
num_tokens_writing_citing

[734]

In [13]:
refined_sources

[{'url': 'https://www.sfbli.com/',
  'refined_text': 'Southern Farm Bureau Life Insurance Company offers a variety of insurance products, specifically focusing on life insurance. Further details on specific types of policies or additional products are not provided in the text.'},
 {'url': 'https://w3.sfbcic.com/',
  'refined_text': 'The text provided does not contain relevant information regarding the insurance products offered by Southern Farm Bureau Life Insurance Company.'},
 {'url': 'https://www.sfbli.com/loginregister',
  'refined_text': 'Southern Farm Bureau Life Insurance Company offers various insurance products, including:\n\n- Life Insurance\n- Variable Life Insurance\n- Health Plans\n- Behavioral Health Plans\n\nThe company aims to provide competitive products and excellent customer service to its policyholders.'},
 {'url': 'https://www.kyfb.com/insurance/insurance-products/life-and-financial/',
  'refined_text': 'Southern Farm Bureau Life Insurance Company offers the follow